In [6]:
import pandas as pd
import numpy as np
import pickle
import tensorflow

In [7]:
GLOVE_TYPE = "twitter"
EMBEDDING_VECTOR_SIZE = 50 # should match glove file
INPUT_LENGTH = 200
N_AUTHORS = 25
VOCAB_SIZE = 5000
df = pd.read_pickle(f"files/data/{GLOVE_TYPE}_{N_AUTHORS}_{EMBEDDING_VECTOR_SIZE}_{INPUT_LENGTH}_df.pickle")

with open(f"files/data/{GLOVE_TYPE}_{N_AUTHORS}_{EMBEDDING_VECTOR_SIZE}_{INPUT_LENGTH}_embedding.pickle",'rb') as f:
    embedding_matrix = pickle.load(f)

## Set up function for building model
Here we use a very basic, somewhat arbitrary LSTM model.

In [63]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Activation, Dense, Embedding
from tensorflow.keras import metrics

def build_lstm_model(vocabulary_size, embedding_dim, input_length, embedding_matrix, output_size):
    ## create model
    model = Sequential()
    model.add(
        Embedding(
            vocabulary_size,
            embedding_dim,
            input_length=input_length,
            weights=[embedding_matrix],
            trainable=False,
        )
    ) 

    model.add(Dense(100, activation="softmax"))

    model.add(Dropout(0.1, noise_shape=None, seed=None))

    model.add(LSTM(output_size, activation="softmax"))

    model.compile(
        loss="categorical_crossentropy",
        optimizer="adam",
        metrics=[metrics.categorical_accuracy, "accuracy"],
    )
    
    #model.summary()
    return model

## Convert author labels into categorical labels

In [64]:
from tensorflow.keras.utils import to_categorical

# need to figure out labeling, unique authors
string_labels = df["author"].unique()
label_dict = {}
for i in range(len(string_labels)):
    label_dict[string_labels[i]] = i
    

labels = df["author"].map(label_dict)
categorical_labels = to_categorical(labels, num_classes=None)

## Format input values

In [65]:
data = df["input"].values.tolist()
data = np.array(data)

## Build model
Use the scikit-learn wrapper so that we can perform cross-validation.

In [67]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from functools import partial

# set up model
model_fn = partial(build_lstm_model, VOCAB_SIZE, EMBEDDING_VECTOR_SIZE, INPUT_LENGTH, embedding_matrix, N_AUTHORS)
model = KerasClassifier(model_fn, epochs=1, batch_size=128)

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
data


kfold = KFold(n_splits=5)
scores = cross_val_score(model, data, categorical_labels, cv=kfold)
scores

Train on 68693 samples
17174/17174 [==============================] - 7s 401us/sample - loss: 2.4946 - categorical_accuracy: 0.2334 - accuracy: 0.2334
Train on 68693 samples
17174/17174 [==============================] - 7s 380us/sample - loss: 2.2781 - categorical_accuracy: 0.4301 - accuracy: 0.4301
Train on 68694 samples
17173/17173 [==============================] - 7s 430us/sample - loss: 2.2454 - categorical_accuracy: 0.4240 - accuracy: 0.4240
Train on 68694 samples
17173/17173 [==============================] - 7s 396us/sample - loss: 2.2974 - categorical_accuracy: 0.4530 - accuracy: 0.4530
Train on 68694 samples
43776/68694 [==================>...........] - ETA: 31s - loss: 3.0076 - categorical_accuracy: 0.2055 - accuracy: 0.2055

array([[ 439,  214,  185, ...,   47,  146, 1170],
       [ 439,  214,  185, ...,   47,  146, 1170],
       [ 439,  214,  185, ...,   47,  146, 1170],
       ...,
       [  11,  384,  969, ...,   10,  631,  169],
       [ 597, 2156, 2350, ...,  670, 1170, 2371],
       [2401,   12, 1008, ..., 2425,  988,  179]])

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)